## 5.- Webserver con ESP8266
En este capitulo se describira como controlar salidas tales como: Motores, Led, servomotores, etc. Entradas tales como: Sensores de temperatura, humedad, distancia, etc. Para cumplir con este fin se programara al ESP8266 como un servidor que permita realizar las acciones. Este programa se denominara webserver.

El programa para el ESP8266 debe:
- Conectarse a Internet.
- Controlar la salidad.
- Programar un webserver que realice las acciones.

Para conectar el ESP8266 a una red WIFI se debe utilizar la libreria  ESP8266Wifi y para construir el webServer la libreria ESP8266WebServer, para acceder a la memoria FLASH se utilizara PROGMEN


Para poder realizar un codigo de programa reusable y facilmente modificable. Se recomienda crear tres archivos.

- internet0.ino: Donde se encuentra el programa principal.
- config.h: Donde se encuentran todas las constantes del programa.
- wifi.hpp: Donde se encuentran las funciones que se utizan para trabajae en Internet.



Se comienza abriendo el IDE arduino. Aparece la siguiente imagen que se muestra en la figura 1

<center><img src="img/wifi0.jpg" alt="Entorno Arduino"></img></center>
<center>Figura 1: IDE Arduino</center>


En la pestaña Archivo  -> se debe seleccionar Guardar como se muestra en la figura 2. Colocando en nombre internet0. Se creo el directorio internet0 y el archivo internet0.ino
    
<center><img src="img/wifi1.jpg" alt="Entorno Arduino"></img></center>
<center>Figura 2: IDE Arduino Guardar archivo</center>

Como se muestra en la figura 3 se realiza un click del raton en los tres puntos señalado por la flecha roja. 
<center><img src="img/wifi2.jpg" alt="Entorno Arduino"></img></center>
<center>Figura 3: IDE Arduino Guardar archivo</center>

Aparece el menu nueva pestaña y se puede crear el archivo config.h y el archivo wifi.hpp ccomo se muestra en la figura 4
<center><img src="img/wifi3.jpg" alt="Entorno Arduino"></img></center>
<center>Figura 4: IDE Arduino </center>

En la figura 4 see muestra como queda el eentorno.
<center><img src="img/wifi4.jpg" alt="Entorno Arduino"></img></center>
<center>Figura 5: IDE Arduino archivso</center>